In [1]:
import pandas as pd
import sqlite3

# Olympic Sports and Medals (ETL): Extract

In [31]:
summer = pd.read_csv("../Data/02_summer.csv")
print(summer.shape)
summer.head()

(31165, 9)


,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


In [32]:
winter = pd.read_csv("../Data/02_winter.csv")
print(winter.shape)
winter.head()

(5770, 9)


,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1924,Chamonix,Biathlon,Biathlon,"BERTHET, G.",FRA,Men,Military Patrol,Bronze
1,1924,Chamonix,Biathlon,Biathlon,"MANDRILLON, C.",FRA,Men,Military Patrol,Bronze
2,1924,Chamonix,Biathlon,Biathlon,"MANDRILLON, Maurice",FRA,Men,Military Patrol,Bronze
3,1924,Chamonix,Biathlon,Biathlon,"VANDELLE, André",FRA,Men,Military Patrol,Bronze
4,1924,Chamonix,Biathlon,Biathlon,"AUFDENBLATTEN, Adolf",SUI,Men,Military Patrol,Gold


In [39]:
winners = pd.concat([summer, winter])
print(winners.shape)
winners.head()

(36935, 9)


,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


In [40]:
winners = winners.drop(['Athlete',
                        'Event',
                        'City',
                        'Gender'], axis=1)
print(winners.shape)
winners.head()

(36935, 5)


,Year,Sport,Discipline,Country,Medal
0,1896,Aquatics,Swimming,HUN,Gold
1,1896,Aquatics,Swimming,AUT,Silver
2,1896,Aquatics,Swimming,GRE,Bronze
3,1896,Aquatics,Swimming,GRE,Gold
4,1896,Aquatics,Swimming,GRE,Silver


In [42]:
winners.describe()

,Year
count,36935.000000
mean,1972.961635
std,32.447833
min,1896.000000
25%,1952.000000
50%,1980.000000
75%,2000.000000
max,2014.000000


In [41]:
winners.Discipline.unique()

array(['Swimming', 'Athletics', 'Cycling Road', 'Cycling Track',
       'Fencing', 'Artistic G.', 'Shooting', 'Tennis', 'Weightlifting',
       'Wrestling Gre-R', 'Water polo', 'Archery', 'Basque Pelota',
       'Cricket', 'Croquet', 'Jumping', 'Football', 'Golf', 'Polo',
       'Rowing', 'Rugby', 'Sailing', 'Tug of War', 'Diving', 'Boxing',
       'Lacrosse', 'Roque', 'Wrestling Free.', 'Hockey', 'Jeu de Paume',
       'Rackets', 'Figure skating', 'Water Motorspor', 'Dressage',
       'Eventing', 'Modern Pentath.', 'Vaulting', 'Ice Hockey',
       'Basketball', 'Canoe / Kayak F', 'Handball', 'Judo', 'Volleyball',
       'Canoe / Kayak S', 'Synchronized S.', 'Rhythmic G.',
       'Table Tennis', 'Badminton', 'Baseball', 'Mountain Bike',
       'Softball', 'Beach volley.', 'Trampoline', 'Taekwondo',
       'Triathlon', 'BMX', 'Marathon swimming', 'Synchronized Swimming',
       'Water Polo', 'Canoe Slalom', 'Canoe Sprint', 'Cycling BMX',
       'Gymnastics Artistic', 'Gymnastics Rhythmi

# Olympic Sports and Medals (ETL): Transform

In [10]:
#creating SQL connection
conn = sqlite3.connect('../Data/olympics.db')
c = conn.cursor()

#function to create table
def create_table(query):
    c.execute(query)

#function to close connection
def close_c_conn():
    c.close()
    conn.close()

In [11]:
#create regions table
create_query = """CREATE TABLE regions
                (NOC TEXT PRIMARY KEY,
                 Region TEXT);"""

c.execute('DROP TABLE IF EXISTS regions')
create_table(create_query)

In [12]:
#function to insert regions into table
def insert_regions(regions):
    for i in range(len(regions.index)):
        c.execute("""INSERT OR REPLACE INTO regions
                    (NOC,
                     Region)
                    VALUES
                   (?,?);""",
                   (regions.iloc[i]['NOC'],
                    regions.iloc[i]['region']))
        
    conn.commit()
    
#insert regions into table
insert_regions(noc_regions)

# Olympic Sports and Medals (ETL): Load

In [13]:
#check SQL regions table
regions = pd.read_sql_query("SELECT * FROM regions;", conn)
print(regions.shape)
regions.head()

(230, 2)


,NOC,Region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [25]:
#close connection
close_c_conn()